In [10]:
import networkx as nx
import csv

file_path_mapping = {
    "CaliAdv": "Dataset\\traj-caliAdv.csv",
    "DisHolly": "Dataset\\traj-disHolly.csv",
    "Disland": "Dataset\\traj-disland.csv",
    "Edin": "Dataset\\traj-Edin.csv",
    "Epcot": "Dataset\\traj-epcot.csv",
    "Glas": "Dataset\\traj-Glas.csv",
    "MagicK": "Dataset\\traj-MagicK.csv",
    "Melb": "Dataset\\traj-Melb.csv",
    "Osak": "Dataset\\traj-Osak.csv",
    "Toro": "Dataset\\traj-Toro.csv"
}

def dataset_parse(pathname):
    trajectory_data = {}

    with open(pathname, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            traj_id = int(row['trajID'])
            poi_id = int(row['poiID'])
            
            if traj_id not in trajectory_data:
                trajectory_data[traj_id] = []
            
            trajectory_data[traj_id].append(poi_id)

    for traj_id, poi_ids in list(trajectory_data.items()):
        if len(poi_ids) < 3:
            del trajectory_data[traj_id]
    
    trajectory_data = list(trajectory_data.items())
    paths = [data[1] for data in trajectory_data]
    weightedG = nx.Graph()
    for path in paths:
        for u, v in zip(path, path[1:]):
            if weightedG.has_edge(u, v):
                weightedG[u][v]['weight'] += 1
            else:
                weightedG.add_edge(u, v, weight=1)

    return (weightedG, paths)

In [11]:
graph, paths = dataset_parse(file_path_mapping["DisHolly"])
# print("Graph: ", graph)
# print("Paths: ", paths)

source = 1
destination = 2
def bfs_shortest_path(graph, start, end):
    queue = [[start]]
    visited = set()

    while queue:
        path = queue.pop(0)
        node = path[-1]

        if node in visited:
            continue

        neighbors = graph.neighbors(node)
        for neighbor in neighbors:
            new_path = list(path)
            new_path.append(neighbor)
            queue.append(new_path)

            if neighbor == end:
                return new_path

        visited.add(node)

    return None

shortest_path = bfs_shortest_path(graph, source, destination)
if shortest_path:
    print(f"Shortest path between {source} and {destination}: {shortest_path}")
else:
    print(f"No path found between {source} and {destination}")

Shortest path between 1 and 2: [1, 2]


In [24]:
import random
graph, paths = dataset_parse(file_path_mapping["DisHolly"])

# Define the source and destination nodes
source = 1
destination = 6

# Perform a random walk to generate a random path
def random_walk(graph, start, end, max_steps=1000):
    path = [start]
    current_node = start

    for _ in range(max_steps):
        neighbors = list(graph.neighbors(current_node))
        valid_neighbors = [neighbor for neighbor in neighbors if neighbor not in path]

        if not valid_neighbors:
            break

        next_node = random.choice(valid_neighbors)
        path.append(next_node)
        current_node = next_node

        if current_node == end:
            return path

    return None

# Generate a random path
random_path = random_walk(graph, source, destination)

def calculate_popularity(graph, random_path):
    edges = [(u, v) for u, v in zip(random_path, random_path[1:])]
    popularity = sum([graph[edge[0]][edge[1]]['weight'] for edge in edges])
    return popularity

if random_path:
    print(f"Random path between {source} and {destination}: {random_path} with popularity {calculate_popularity(graph, random_path)}")
else:
    print(f"No path found between {source} and {destination}")

Random path between 1 and 6: [1, 12, 13, 7, 10, 3, 2, 9, 11, 8, 6] with popularity 263
